# Part 1 Converts output of OpenPose (.json) to a .txt file
## Part 1.1 Import required libraries

In [1]:
import json
from pprint import pprint
import glob, os
import numpy as np
from datetime import date

## Part 1. 2 Extract Keypoints
Extract x, y co-ordinates for all pose keypoints and all frames. **This code can handle only one video at a time i.e. keypoints from one test video can be used per run.** 

In [11]:
# Change the data path where all the .json files are
data_path = r"data/MHAD/Test/"
os.chdir(data_path)
print(os.getcwd())

kps = []
openpose_2person_count = 0

for file in sorted(glob.glob("*.json")):
    with open(file) as data_file: 
        data = json.load(data_file)
        if len(data["people"]) > 1:
            pprint("More than one detection in file, check the noise:")
            openpose_2person_count += 1
            print(file)
        frame_kps = []
        pose_keypoints = data["people"][0]["pose_keypoints_2d"]
        j = 0
        for i in range(36):
            frame_kps.append(pose_keypoints[j])
            j += 1
            if (j+1)%3 == 0:
                j += 1
        kps.append(frame_kps)

C:\Users\BV SAMEER KUMAR\Documents\JupterNotebookCodes\output


In [12]:
# Check the shape of the data
kps_np = np.array(kps)
print(kps_np.shape)
print(len(kps))

(95, 36)
95


In [13]:
# Check how many frames contained more than 1 person 
print(openpose_2person_count)

0


## Part 1.3 Create a .txt file
Write all the data into a .txt file. Each row indicates all the frames that were extracted, and in each row we will have 36 entries corresponding to x, y postions for all 18 keypoints (joints). 

In [14]:
# Change the data path where .txt file will be store for testing
data_path = r"data/MHAD/database/"
os.chdir(data_path)
print(os.getcwd())

file_name = "testvid_" + str(date.today()) + ".txt"
with open(file_name, "w") as text_file:
    for i in range(len(kps)):
        for j in range(36):
            text_file.write('{}'.format(kps[i][j]))
            if j < 35:
                text_file.write(',')
        text_file.write('\n')

# Part 2 Convert .txt file into a testing data point

In [15]:
# num_steps depends on rate of video and window_width to be used
# in this case camera was 22Hz and a window_width of 1.5s was wanted, giving 22*1.5 = 33
num_steps = 32
overlap = 0.8125 # 0 = 0% overlap, 1 = 100%

In [17]:
count = 0
for file in sorted(glob.glob("*.txt")):
    data_file = open(file,'r')
    file_text = data_file.readlines() 
    num_frames = len(file_text)
    num_framesets = int((num_frames - num_steps)/(num_steps*(1-overlap)))+1
    data_file.close
    file_name = "datapoint_" + str(count) + "_testvid_" + str(date.today()) + ".txt"
    x_file = open(file_name, 'a')
    for frameset in range(0, num_framesets):
        start = int(frameset*num_steps*(1-overlap))
        for line in range(start,(start+num_steps)):
            x_file.write(file_text[line])
    x_file.close()